# Bidding

This is a tutorial for how to use the bidding engine

In [1]:
import sys
sys.path.append('../../src')

import os
import logging

# Set logging level to suppress warnings
logging.getLogger().setLevel(logging.ERROR)
# Just disables the warnings
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '2'
from nn.models import Models
from bots import BotBid
from bidding import bidding
from util import hand_to_str
from deck52 import random_deal
from sample import Sample
import conf
import numpy as np

np.set_printoptions(precision=2, suppress=True, linewidth=200)
np.random.seed(42)

Instructions for updating:
non-resource variables are not supported in the long term


In [2]:
models = Models.from_conf(conf.load('../Conf/UCBC2024.conf'),'..')   # loading neural networks
sampler = Sample.from_conf(conf.load('../Conf/UCBC2024.conf'), False)  # Load sampling strategies

INFO:tensorflow:Restoring parameters from ..\Models/NS1EW99-bidding-10724000
INFO:tensorflow:Restoring parameters from ..\Models/NS1EW99-binfo-6131000
INFO:tensorflow:Restoring parameters from ..\Models/lead_suit-154000
INFO:tensorflow:Restoring parameters from ..\Models/lead_nt-59000
INFO:tensorflow:Restoring parameters from ..\Models/lr3-1000000
INFO:tensorflow:Restoring parameters from ..\Models/single_dummy-32768000
INFO:tensorflow:Restoring parameters from ..\Models/lefty-1000000
INFO:tensorflow:Restoring parameters from ..\Models/dummy-920000
INFO:tensorflow:Restoring parameters from ..\Models/righty-1000000
INFO:tensorflow:Restoring parameters from ..\Models/decl-1000000


### Bidding through an entire auction for all four hands

Now we'll take a random deal and let the bot bid for each hand. We assume North deals and nobody is vulnerable.

This script was used to detect the reason for some strange bidding. But the error is fixed, so this should look normal now

In [3]:
hands = ['T.A2.QT9763.QJ97','98743.KJ3.J52.A8','KJ52.9854.A.K653','AQ6.QT76.K84.T42']

In [4]:
bidder_bots = [BotBid([True, False], hand, models, sampler, i, False) for i, hand in enumerate(hands)]

In [6]:
auction = ['PAD_START','PAD_START']  # since West deals we need 'PAD_START'

turn_i = len(auction) % 4 # whose turn is it to bid
step = 0

while not bidding.auction_over(auction):
    bid = bidder_bots[turn_i].bid(auction)
    # if step== 1:
    # bid.bid='X'
    if step== 3:
        bid.bid='1S'
    if step== 5:
        bid.bid='2C'
    print(bid.to_dict())
    auction.append(bid.bid)
    turn_i = (turn_i + 1) % 4  # next player's turn
    step += 1
print(auction)

['PAD_START', 'PAD_START']
Bid not valid: PAD_END insta_score: 0.997 0.03
['PAD_START', 'PAD_START', 'PASS']
Bid not valid: PAD_END insta_score: 0.999 0.03
['PAD_START', 'PAD_START', 'PASS', 'PASS']
Bid not valid: PAD_END insta_score: 1.000 0.03
['PAD_START', 'PAD_START', 'PASS', 'PASS', 'PASS']
Bid not valid: PAD_END insta_score: 1.000 0.03
['PAD_START', 'PAD_START', 'PASS', 'PASS', 'PASS', '1S']
Bid not valid: PAD_END insta_score: 0.076 0.03


{'bid': 'PASS', 'candidates': [{'call': 'PASS', 'insta_score': 0.003}], 'samples': [], 'hcp': -1, 'shape': -1, 'who': 'NN'}
{'bid': 'PASS', 'candidates': [{'call': 'PASS', 'insta_score': 0.001}], 'samples': [], 'hcp': -1, 'shape': -1, 'who': 'NN'}
{'bid': 'PASS', 'candidates': [{'call': 'PASS', 'insta_score': 0.0}], 'samples': [], 'hcp': -1, 'shape': -1, 'who': 'NN'}
{'bid': '1S', 'candidates': [{'call': 'PASS', 'insta_score': 0.0}], 'samples': [], 'hcp': -1, 'shape': -1, 'who': 'NN'}
{'bid': 'PASS', 'candidates': [{'call': 'PASS', 'insta_score': 0.768, 'expected_score': -95, 'adjustment': 38}, {'call': 'X', 'insta_score': 0.138, 'expected_score': -159, 'adjustment': 7}], 'samples': ['9x.JTxxx.QJx.Qxx AQ8xx.A.K8xx.JTx KJxx.98xx.A.Kxxx Tx.KQx.T9xxx.A98 0.99985 \n P-P-P-1S-P-1N-P-2D-P-3D-P-P-P (9.4)  \n P-P-P-1S-X-1N-2H-P-P-3D-P-P-3H-P-P-P (7.2) ', 'Tx.KQxx.J8xx.J8x A98xx.xx.KT9x.Ax KJxx.98xx.A.Kxxx Qx.AJT.Qxxx.QT9x 0.99573 \n P-P-P-1S-P-1N-P-2D-P-3D-P-P-P (8.9)  \n P-P-P-1S-X-XX-2H-P-P-

['PAD_START', 'PAD_START', 'PASS', 'PASS', 'PASS', '1S', 'PASS']
Bid not valid: PAD_END insta_score: 0.363 0.03
['PAD_START', 'PAD_START', 'PASS', 'PASS', 'PASS', '1S', 'PASS', '2C']
Bid not valid: PAD_END insta_score: 0.039 0.03
['PAD_START', 'PAD_START', 'PASS', 'PASS', 'PASS', '1S', 'PASS', '2C', 'PASS']
Bid not valid: PAD_END insta_score: 1.000 0.03


{'bid': 'PASS', 'candidates': [{'call': 'PASS', 'insta_score': 0.867, 'expected_score': 37, 'adjustment': 433}, {'call': 'X', 'insta_score': 0.109, 'expected_score': -54, 'adjustment': 55}], 'samples': ['A.xxx.T98xxxxx.x QT9xx.KTx.KQ.A98 KJxx.98xx.A.Kxxx 8xx.AQJ.Jx.QJTxx 0.02501 \n P-P-P-1S-P-2C-P-P-P (7.4)  \n P-P-P-1S-P-2C-P-P-X-P-P-P (7.4) ', '9.AKTx.98xxxx.Qx QT8xx.Qx.KQx.JTx KJxx.98xx.A.Kxxx Axx.Jxx.JTx.A98x 0.02228 \n P-P-P-1S-P-2C-P-P-P (5.5)  \n P-P-P-1S-P-2C-P-P-X-P-2D-2S-P-P-3D-P-P-3S-P-P-P (6.3) ', '.QTx.QT8xxxx.Jxx AQ98x.AJx.Jx.Q8x KJxx.98xx.A.Kxxx Txxx.Kxx.K9x.AT9 0.02178 \n P-P-P-1S-P-2C-P-P-P (7.1)  \n P-P-P-1S-P-2C-P-P-X-P-2D-P-P-2S-P-P-P (9.8) ', 'x.QJxx.Qxxxxx.Jx AQ8xx.ATx.K8.T8x KJxx.98xx.A.Kxxx T9x.Kx.JT9x.AQ9x 0.01503 \n P-P-P-1S-P-2C-P-P-P (8.3)  \n P-P-P-1S-P-2C-P-P-X-P-2D-P-P-2S-P-P-P (9.4) ', 'T9.QJxxx.Txxx.Jx AQxxx.Kx.Kxx.Q98 KJxx.98xx.A.Kxxx 8x.AT.QJ98x.ATxx 0.01483 \n P-P-P-1S-P-2C-P-P-P (9.0)  \n P-P-P-1S-P-2C-P-P-X-P-2H-P-P-P (7.2) ', 'x.Kxx.Q98xx.QJxx AQT